In [ ]:
# jupyter_vim
# docker-compose-cdc.yaml automatically spins up containers (see generator service) that publishes messages to the postgres database. Sequin stream cdc to kafka topic 'cdc'. See the sequin service at localhost:7376

In [1]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="postgres", port=5432, user="admin", password="admin", dbname="cdc"
)

In [2]:
with conn.cursor() as cur:
    cur.execute(f"SELECT count(*) FROM messages;")
    print(cur.fetchall())
    cur.execute(f"SELECT * FROM messages LIMIT 20;")
    values = cur.fetchall()
df = pd.DataFrame(
    values,
    columns=[
        "event_id",
        "origin_id",
        "message",
        "created_at",
        "inserted_at",
        "updated_at",
    ],
)
df.tail(5)

[(84,)]


,event_id,origin_id,message,created_at,inserted_at,updated_at
15,f168f80261e048808a634dc68f2ce283,70e7d3fd6ee2499cb161b2211a0e2261,Can expert feel believe tax field pick boy acc...,2025-03-18 19:13:21,2025-03-18 19:13:21.814118,2025-03-18 19:13:21.814118
16,c21d2fa106514be6b3cbf92a2cdf55b8,70e7d3fd6ee2499cb161b2211a0e2261,Act yourself raise machine them far manager ok...,2025-03-18 19:13:22,2025-03-18 19:13:22.815607,2025-03-18 19:13:22.815607
17,c08293ce92ff439eadb8c617c44da6b3,70e7d3fd6ee2499cb161b2211a0e2261,Easy poor nation network successful mother pro...,2025-03-18 19:13:23,2025-03-18 19:13:23.817098,2025-03-18 19:13:23.817098
18,56d6c1609d73481eb4d328af3e99aa86,70e7d3fd6ee2499cb161b2211a0e2261,Investment nature public easy out share night ...,2025-03-18 19:13:24,2025-03-18 19:13:24.819191,2025-03-18 19:13:24.819191
19,4e4d87fde7904b31871a544baf3380ac,70e7d3fd6ee2499cb161b2211a0e2261,Per drive investment cultural possible enter p...,2025-03-18 19:13:25,2025-03-18 19:13:25.820894,2025-03-18 19:13:25.820894


In [3]:
from kafka import KafkaConsumer, TopicPartition
from kafka.admin import KafkaAdminClient, ConfigResource, ConfigResourceType


bootstrap_servers = "kafka-broker:9092"

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id="demo",
)
topic = "cdc"

In [12]:
import json

consumer_topic = KafkaConsumer(
    topic,
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
    consumer_timeout_ms=5000,
)


In [13]:
# Poll with a timeout (in milliseconds)
timeout_ms = 5000
while True:
    # Poll for messages
    message_batch = consumer_topic.poll(timeout_ms=timeout_ms)
    
    # If no messages received, break the loop
    if not message_batch:
        break
    
    # Process the messages
    for partition_messages in message_batch.values():
        for message in partition_messages:
            decoded = json.loads(message.value.decode("utf-8"))
            if decoded["action"] == "insert":
                inserts.append(decoded)
            elif decoded["action"] == "delete":
                deletions.append(decoded)
            else:
                print(decoded)
            records.append(decoded)
consumer_topic.close()

In [14]:
print("number of records:", len(records))
print("number of deletions:", len(deletions))
print("number of insertions:", len(inserts))

number of records: 360
number of deletions: 0
number of insertions: 360


In [15]:
def quick_topic_count(topic_name, bootstrap_servers=["kafka-broker:9092"]):
    consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers)

    # Get partition information
    partitions = consumer.partitions_for_topic(topic_name)
    if not partitions:
        return 0

    # Create TopicPartition objects
    tps = [TopicPartition(topic_name, p) for p in partitions]

    # Get end offsets (latest messages)
    end_offsets = consumer.end_offsets(tps)

    # Get beginning offsets (earliest messages)
    beginning_offsets = consumer.beginning_offsets(tps)

    # Calculate total
    total = sum(end_offsets[tp] - beginning_offsets[tp] for tp in tps)

    consumer.close()
    return total

In [16]:
quick_topic_count("cdc")

120